In [ ]:
# E-commerce Feature Engineering  

## Project Overview
"""
This notebook focuses on advanced feature engineering techniques to create meaningful features for machine learning models and business analysis.

### Feature Engineering Categories:
    "1. **Temporal Features** - Time-based patterns and trends  
    "2. **Customer Behavioral Features** - RFM analysis and customer patterns  
    "3. **Product Features** - Product performance and characteristics  
    "4. **Transactional Features** - Purchase patterns and basket analysis  
    "5. **Geographic Features** - Location-based insights  
    "6. **Advanced ML Features** - Features for predictive modeling  
    "7. **Feature Selection & Validation** - Identifying most important features"
 """
 # Import libraries
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns  
from datetime import datetime, timedelta  
from sklearn.preprocessing import StandardScaler, LabelEncoder  
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression  
import warnings  
warnings.filterwarnings('ignore')  
 
# Setup  
plt.style.use('seaborn-v0_8')  
sns.set_palette("husl")  
pd.set_option('display.max_columns', None)  
print("🔧 Feature Engineering Started...")
# Load processed data  
print("📂 Loading data...")  
transactions = pd.read_csv('../data/processed/enriched_transactions.csv')  
customers = pd.read_csv('../data/processed/customers_clean.csv')  
products = pd.read_csv('../data/processed/products_clean.csv')  

# Convert date columns  
transactions['date'] = pd.to_datetime(transactions['date'])  
customers['signup_date'] = pd.to_datetime(customers['signup_date'])  
print(f"✅ Data loaded: Transactions {transactions.shape}, Customers {customers.shape}, Products {products.shape}") 
## 1. Temporal Feature Engineering"
print("⏰ TEMPORAL FEATURE ENGINEERING")  
print("=" * 50)  
 
# Extract comprehensive time-based features  
transactions_fe = transactions.copy()  
 
# Basic date features  
transactions_fe['year'] = transactions_fe['date'].dt.year  
transactions_fe['month'] = transactions_fe['date'].dt.month  
transactions_fe['quarter'] = transactions_fe['date'].dt.quarter  
transactions_fe['week'] = transactions_fe['date'].dt.isocalendar().week  
transactions_fe['day'] = transactions_fe['date'].dt.day  
transactions_fe['day_of_week'] = transactions_fe['date'].dt.dayofweek  
transactions_fe['day_of_year'] = transactions_fe['date'].dt.dayofyear  
transactions_fe['is_weekend'] = transactions_fe['date'].dt.dayofweek >= 5  
transactions_fe['is_month_start'] = transactions_fe['date'].dt.is_month_start  
transactions_fe['is_month_end'] = transactions_fe['date'].dt.is_month_end  

# Seasonal features  
transactions_fe['season'] = transactions_fe['month'].apply(  
    lambda x: 'Winter' if x in [12, 1, 2] else  
                 'Spring' if x in [3, 4, 5] else  
                 'Summer' if x in [6, 7, 8] else 'Fall'  
    )  

# Holiday periods (simplified)  
def get_holiday_period(month, day):  
    if month == 12 and day >= 15:  # Christmas season  
        return 'Holiday'  
    elif month == 1 and day <= 7:  # New Year  
        return 'Holiday'  
    elif month == 7 and day == 4:  # Independence Day  
        return 'Holiday'  
    else:  
        return 'Regular'  
   
transactions_fe['holiday_period'] = transactions_fe.apply(  
    lambda x: get_holiday_period(x['month'], x['day']), axis=1  
    )  
   
# Time of day features (assuming we had hour data)  
# For now, we'll create synthetic time patterns  
np.random.seed(42)  
transactions_fe['hour'] = np.random.randint(0, 24, len(transactions_fe))  
transactions_fe['time_of_day'] = pd.cut(transactions_fe['hour'],   
                                       bins=[0, 6, 12, 18, 24],   
                                       labels=['Night', 'Morning', 'Afternoon', 'Evening'],  
                                       include_lowest=True)  
print("✅ Temporal features created:")  
print(f"   • Date components (year, month, quarter, week, day)")  
print(f"   • Day of week and year")  
print(f"   • Weekend and month boundary flags")  
print(f"   • Seasonal classification")  
print(f"   • Holiday period identification")  
print(f"   • Time of day segments")
# Analyze temporal patterns  
print("📊 Temporal Feature Analysis:")  
 
fig, axes = plt.subplots(2, 2, figsize=(15, 10))  
 
# Revenue by season  
season_revenue = transactions_fe.groupby('season')['revenue'].sum()  
axes[0,0].bar(season_revenue.index, season_revenue.values, color=['lightblue', 'lightgreen', 'gold', 'lightcoral'])  
axes[0,0].set_title('Revenue by Season')  
axes[0,0].set_ylabel('Total Revenue ($)')  

# Transactions by day of week  
dow_transactions = transactions_fe.groupby('day_of_week')['transaction_id'].count()  
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']  
axes[0,1].bar(day_names, dow_transactions.values, color='skyblue')  
axes[0,1].set_title('Transactions by Day of Week')  
axes[0,1].set_ylabel('Number of Transactions')  

# Revenue by holiday period  
holiday_revenue = transactions_fe.groupby('holiday_period')['revenue'].sum()  
axes[1,0].pie(holiday_revenue.values, labels=holiday_revenue.index, autopct='%1.1f%%')  
axes[1,0].set_title('Revenue Distribution: Holiday vs Regular')  

# Transactions by time of day  
tod_transactions = transactions_fe.groupby('time_of_day')['transaction_id'].count()  
axes[1,1].bar(tod_transactions.index, tod_transactions.values, color=['navy', 'blue', 'lightblue', 'skyblue'])  
axes[1,1].set_title('Transactions by Time of Day')  
axes[1,1].set_ylabel('Number of Transactions')  

plt.tight_layout()  
plt.show()
## 2. Customer Behavioral Features"
print("👥 CUSTOMER BEHAVIORAL FEATURE ENGINEERING")  
print("=" * 50)  
   
# Calculate customer-level features  
customer_features = transactions_fe.groupby('customer_id').agg({  
    'revenue': ['sum', 'mean', 'std', 'count'],  
    'quantity': ['sum', 'mean', 'std'],  
    'date': ['min', 'max', 'nunique'],  
    'product_id': 'nunique',  
    'category': 'nunique'  
    }).round(2)  
   
# Flatten column names  
customer_features.columns = [  
    'total_revenue', 'avg_revenue', 'std_revenue', 'transaction_count',  
    'total_quantity', 'avg_quantity', 'std_quantity',  
    'first_purchase', 'last_purchase', 'unique_days',  
    'unique_products', 'unique_categories'
    ]  
   
customer_features = customer_features.reset_index()  
   
# Calculate additional customer metrics  
latest_date = transactions_fe['date'].max()  
customer_features['customer_lifetime_days'] = (latest_date - customer_features['first_purchase']).dt.days  
customer_features['days_since_last_purchase'] = (latest_date - customer_features['last_purchase']).dt.days  
customer_features['purchase_frequency'] = customer_features['transaction_count'] / customer_features['customer_lifetime_days']  
customer_features['avg_days_between_purchases'] = customer_features['customer_lifetime_days'] / customer_features['transaction_count']  

# RFM Scoring  
customer_features['recency_score'] = pd.qcut(customer_features['days_since_last_purchase'], 5, labels=[5, 4, 3, 2, 1])  
customer_features['frequency_score'] = pd.qcut(customer_features['transaction_count'], 5, labels=[1, 2, 3, 4, 5])  
customer_features['monetary_score'] = pd.qcut(customer_features['total_revenue'], 5, labels=[1, 2, 3, 4, 5])  
customer_features['rfm_score'] = (  
    customer_features['recency_score'].astype(str) +   
    customer_features['frequency_score'].astype(str) +   
    customer_features['monetary_score'].astype(str)  
    )  

# Customer segmentation based on RFM  
def segment_customer(row):  
    r, f, m = row['recency_score'], row['frequency_score'], row['monetary_score']  
    
    if r >= 4 and f >= 4 and m >= 4:  
        return 'Champions'  
    elif r >= 3 and f >= 3 and m >= 3:  
        return 'Loyal Customers'  
    elif r >= 4 and f <= 2:  
        return 'New Customers'  
    elif r >= 2 and f >= 2:  
        return 'Potential Loyalists'  
    elif r <= 2 and f >= 3:  
        return 'At Risk'  
    elif r <= 2 and f <= 2:  
        return 'Lost Customers'  
    else:  
        return 'Regular Customers'  
    
customer_features['customer_segment'] = customer_features.apply(segment_customer, axis=1)  

# Customer value tier  
customer_features['value_tier'] = pd.qcut(customer_features['total_revenue'],   
                                         q=4,   
                                         labels=['Bronze', 'Silver', 'Gold', 'Platinum'])  
   
print("✅ Customer behavioral features created:")  
print(f"   • Basic spending metrics (total, average, std)")  
print(f"   • Purchase frequency and recency")  
print(f"   • Product and category diversity")  
print(f"   • RFM scoring and segmentation")  
print(f"   • Customer value tiers")  
print(f" 📊 Customer Segments Distribution:")  
print(customer_features['customer_segment'].value_counts())
# Visualize customer segments  
fig, axes = plt.subplots(2, 2, figsize=(15, 10))  
 
# Customer segments distribution  
segment_dist = customer_features['customer_segment'].value_counts()  
axes[0,0].pie(segment_dist.values, labels=segment_dist.index, autopct='%1.1f%%', startangle=90)  
axes[0,0].set_title('Customer Segments Distribution')  

# Average revenue by segment  
segment_revenue = customer_features.groupby('customer_segment')['total_revenue'].mean().sort_values(ascending=False)  
axes[0,1].bar(segment_revenue.index, segment_revenue.values, color='lightgreen')  
axes[0,1].set_title('Average Revenue by Customer Segment')  
axes[0,1].set_ylabel('Average Revenue ($)')  
axes[0,1].tick_params(axis='x', rotation=45)  
 
# Purchase frequency by segment  
segment_frequency = customer_features.groupby('customer_segment')['purchase_frequency'].mean().sort_values(ascending=False)  
axes[1,0].bar(segment_frequency.index, segment_frequency.values, color='lightcoral')  
axes[1,0].set_title('Average Purchase Frequency by Segment')  
axes[1,0].set_ylabel('Purchases per Day')  
axes[1,0].tick_params(axis='x', rotation=45)  
 
# Customer lifetime by segment  
segment_lifetime = customer_features.groupby('customer_segment')['customer_lifetime_days'].mean().sort_values(ascending=False)  
axes[1,1].bar(segment_lifetime.index, segment_lifetime.values, color='gold')  
axes[1,1].set_title('Average Customer Lifetime by Segment')  
axes[1,1].set_ylabel('Lifetime (Days)')  
axes[1,1].tick_params(axis='x', rotation=45)  
 
plt.tight_layout()  
plt.show()  
## 3. Product Feature Engineering"
print("📦 PRODUCT FEATURE ENGINEERING")  
print("=" * 50)  
 
# Product performance features  
product_features = transactions_fe.groupby('product_id').agg({  
    'revenue': ['sum', 'mean', 'std', 'count'],  
    'quantity': ['sum', 'mean', 'std'],  
    'profit': ['sum', 'mean'],  
    'customer_id': 'nunique',  
    'date': ['min', 'max', 'nunique']  
    }).round(2)  
     
# Flatten column names  
product_features.columns = [  
    'total_revenue', 'avg_revenue_per_tx', 'std_revenue', 'transaction_count',  
    'total_quantity', 'avg_quantity_per_tx', 'std_quantity',  
    'total_profit', 'avg_profit_per_tx',  
    'unique_customers',  
    'first_sale', 'last_sale', 'unique_sale_days'  
    ]  
     
product_features = product_features.reset_index()  
    
# Merge with product master data  
product_features = product_features.merge(  
    products[['product_id', 'product_name', 'category', 'price', 'price_segment', 'profit_margin']],  
    on='product_id',  
    how='left'  
    )  
      
# Calculate additional product metrics  
product_features['product_lifetime_days'] = (latest_date - product_features['first_sale']).dt.days  
product_features['sales_velocity'] = product_features['total_quantity'] / product_features['product_lifetime_days']  
product_features['customer_penetration_rate'] = product_features['unique_customers'] / len(customer_features)  
product_features['avg_days_between_sales'] = product_features['product_lifetime_days'] / product_features['transaction_count']  
product_features['profit_margin_actual'] = (product_features['total_profit'] / product_features['total_revenue']) * 100  
 
# Product popularity score  
product_features['popularity_score'] = (  
    product_features['total_quantity'] / product_features['total_quantity'].max() * 0.4 +  
    product_features['unique_customers'] / product_features['unique_customers'].max() * 0.3 +  
    product_features['transaction_count'] / product_features['transaction_count'].max() * 0.3  
    ) * 100  
     
# Product performance tier  
product_features['performance_tier'] = pd.qcut(product_features['total_revenue'],   
                                              q=4,   
                                              labels=['Low', 'Medium', 'High', 'Premium'])  
 
# Seasonality indicator (simplified)  
product_seasonality = transactions_fe.groupby(['product_id', 'season'])['quantity'].sum().unstack(fill_value=0)  
product_seasonality['seasonality_score'] = product_seasonality.std(axis=1) / product_seasonality.mean(axis=1)  
product_features = product_features.merge(  
    product_seasonality[['seasonality_score']],   
    on='product_id',   
    how='left'  
    )  
     
print("✅ Product features created:")  
print(f"   • Sales performance metrics")  
print(f"   • Customer reach and penetration")  
print(f"   • Sales velocity and frequency")  
print(f"   • Profitability analysis")  
print(f"   • Popularity scoring")  
print(f"   • Performance tier classification")  
print(f"   • Seasonality patterns")  
print(f"📊 Top 10 Products by Popularity Score:")  
top_products = product_features.nlargest(10, 'popularity_score')[['product_name', 'category', 'popularity_score', 'total_revenue']]  
display(top_products)
## 4. Transactional & Basket Analysis Features"
print("🛒 TRANSACTIONAL & BASKET ANALYSIS FEATURES")  
print("=" * 50)  
 
# Transaction-level features  
transaction_features = transactions_fe.groupby('transaction_id').agg({  
    'quantity': 'sum',  
    'revenue': 'sum',  
    'profit': 'sum',  
    'product_id': 'count',  
    'category': 'nunique'  
    }).round(2)  
     
transaction_features.columns = [  
    'basket_size', 'basket_value', 'basket_profit',   
    'unique_products', 'unique_categories'  
]  
    
transaction_features = transaction_features.reset_index()  
    
# Merge back with transaction details  
transaction_features = transaction_features.merge(  
    transactions_fe[['transaction_id', 'customer_id', 'date', 'region', 'customer_tier']].drop_duplicates(),  
    on='transaction_id',  
    how='left'  
)  

# Calculate basket metrics  
transaction_features['avg_product_value'] = transaction_features['basket_value'] / transaction_features['unique_products']  
transaction_features['basket_efficiency'] = transaction_features['basket_profit'] / transaction_features['basket_value']  
transaction_features['category_diversity'] = transaction_features['unique_categories'] / transaction_features['unique_products']  

# Basket type classification  
def classify_basket(row):  
    if row['unique_products'] == 1:  
        return 'Single Item'  
    elif row['unique_products'] <= 3:  
        return 'Small Basket'  
    elif row['unique_products'] <= 6:  
        return 'Medium Basket'  
    else:  
        return 'Large Basket'  

transaction_features['basket_type'] = transaction_features.apply(classify_basket, axis=1)  

# Value-based basket classification  
transaction_features['value_segment'] = pd.qcut(transaction_features['basket_value'],   
                                                   q=4,   
                                                   labels=['Low Value', 'Medium Value', 'High Value', 'Premium Value'])  

# Cross-selling opportunities (simplified)  
# For each transaction, identify if it contains products from multiple categories  
transaction_features['is_cross_category'] = transaction_features['unique_categories'] > 1  

print("✅ Transactional features created:")  
print(f"   • Basket size and value metrics")  
print(f"   • Product and category diversity")  
print(f"   • Basket efficiency (profitability)")  
print(f"   • Basket type classification")  
print(f"   • Value-based segmentation")  
print(f"   • Cross-category purchase indicators")  
print(f"📊 Basket Analysis Summary:")  
basket_summary = transaction_features.groupby('basket_type').agg({  
    'transaction_id': 'count',  
    'basket_value': 'mean',  
    'basket_size': 'mean',  
    'basket_efficiency': 'mean'  
    }).round(2)  
    
basket_summary.columns = ['transaction_count', 'avg_basket_value', 'avg_basket_size', 'avg_efficiency']  
display(basket_summary)
## 5. Geographic Feature Engineering"
print("🌎 GEOGRAPHIC FEATURE ENGINEERING")  
print("=" * 50)  
 
# Regional performance features  
regional_features = transactions_fe.groupby('region').agg({  
    'revenue': ['sum', 'mean', 'std'],  
    'profit': ['sum', 'mean'],  
    'transaction_id': 'count',  
    'customer_id': 'nunique',  
    'quantity': 'sum'  
    }).round(2)  
     
# Flatten column names  
regional_features.columns = [  
    'total_revenue', 'avg_revenue_per_tx', 'std_revenue',  
    'total_profit', 'avg_profit_per_tx',  
    'transaction_count',  
    'unique_customers',  
    'total_quantity'  
]  

regional_features = regional_features.reset_index()  
 
# Calculate regional metrics  
total_customers = len(customer_features)  
total_revenue = regional_features['total_revenue'].sum()  

regional_features['market_share'] = (regional_features['total_revenue'] / total_revenue) * 100  
regional_features['customer_penetration'] = (regional_features['unique_customers'] / total_customers) * 100  
regional_features['revenue_per_customer'] = regional_features['total_revenue'] / regional_features['unique_customers']  
regional_features['transactions_per_customer'] = regional_features['transaction_count'] / regional_features['unique_customers']  
regional_features['avg_transaction_value'] = regional_features['total_revenue'] / regional_features['transaction_count']  

# Regional performance tier  
regional_features['performance_tier'] = pd.qcut(regional_features['total_revenue'],   
    q=3,   
    labels=['Low', 'Medium', 'High'])  
   
# Regional efficiency score  
regional_features['efficiency_score'] = (  
    regional_features['market_share'] / regional_features['market_share'].max() * 0.4 +  
    regional_features['revenue_per_customer'] / regional_features['revenue_per_customer'].max() * 0.3 +  
    regional_features['customer_penetration'] / regional_features['customer_penetration'].max() * 0.3  
    ) * 100  
    
print("✅ Geographic features created:")  
print(f"   • Regional sales performance")  
print(f"   • Market share and penetration")  
print(f"   • Customer value metrics")  
print(f"   • Performance tier classification")  
print(f"   • Regional efficiency scoring")  
print(f"📊 Regional Performance Summary:")  
display(regional_features.sort_values('efficiency_score', ascending=False))
## 6. Advanced ML Features & Feature Selection"
print("🤖 ADVANCED ML FEATURES & FEATURE SELECTION")  
print("=" * 50)  
 
# Create a comprehensive feature set for machine learning  
# We'll create features at customer level for prediction tasks  
 
# Start with customer features  
ml_features = customer_features.copy()  
 
# Add temporal behavior features  
customer_temporal = transactions_fe.groupby('customer_id').agg({  
    'day_of_week': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 0,  # Most frequent purchase day  
        'season': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown',  # Most frequent season  
        'is_weekend': 'mean',  # Weekend purchase ratio  
        'hour': ['mean', 'std']  # Purchase time patterns  
    }).round(2)  
     
customer_temporal.columns = ['preferred_day', 'preferred_season', 'weekend_ratio', 'avg_purchase_hour', 'std_purchase_hour']  
customer_temporal = customer_temporal.reset_index()  
    
ml_features = ml_features.merge(customer_temporal, on='customer_id', how='left')  
    
# Add product preference features  
customer_preferences = transactions_fe.groupby('customer_id').agg({  
    'category': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown',  # Favorite category  
    'price_segment': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown',  # Preferred price segment  
    'profit_margin': 'mean'  # Average margin of purchased products  
    }).round(2)  
    
customer_preferences.columns = ['preferred_category', 'preferred_price_segment', 'avg_product_margin']  
customer_preferences = customer_preferences.reset_index()  
    
ml_features = ml_features.merge(customer_preferences, on='customer_id', how='left')  
    
# Add regional features  
customer_region = transactions_fe.groupby('customer_id').agg({  
    'region': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown'  # Primary region  
    })  
customer_region.columns = ['primary_region']  
customer_region = customer_region.reset_index()  
    
ml_features = ml_features.merge(customer_region, on='customer_id', how='left')  
    
# Add basket behavior features  
customer_basket = transaction_features.groupby('customer_id').agg({  
    'basket_value': ['mean', 'std', 'max'],  
    'basket_size': ['mean', 'std'],  
    'basket_efficiency': 'mean',  
    'is_cross_category': 'mean'  
    }).round(2)  
    
customer_basket.columns = [  
    'avg_basket_value', 'std_basket_value', 'max_basket_value',  
    'avg_basket_size', 'std_basket_size',  
    'avg_basket_efficiency',  
    'cross_category_ratio'  
]  
customer_basket = customer_basket.reset_index()  
    
ml_features = ml_features.merge(customer_basket, on='customer_id', how='left')  
    
# Create target variable for predictive modeling (e.g., high-value customer)  
ml_features['is_high_value'] = (ml_features['total_revenue'] > ml_features['total_revenue'].quantile(0.75)).astype(int)  
    
# Create churn risk indicator (customers with no recent purchases)  
ml_features['churn_risk'] = (ml_features['days_since_last_purchase'] > 90).astype(int)  
    
print("✅ Advanced ML features created:")  
print(f"   • Temporal behavior patterns")  
print(f"   • Product preferences")  
print(f"   • Regional patterns")  
print(f"   • Basket behavior metrics")  
print(f"   • Target variables for ML")  
print(f"   • Churn risk indicators")  
print(f"📊 ML Dataset Shape: {ml_features.shape}")  
print(f"📊 High-value customers: {ml_features['is_high_value'].sum()} ({ml_features['is_high_value'].mean()*100:.1f}%)")  
print(f"📊 At-risk customers: {ml_features['churn_risk'].sum()} ({ml_features['churn_risk'].mean()*100:.1f}%)")
# Feature importance analysis  
print("🎯 FEATURE IMPORTANCE ANALYSIS")  
print("=" * 50)  
 
# Select numerical features for importance analysis  
numerical_features = ml_features.select_dtypes(include=[np.number]).columns.tolist()  
# Remove target variables and ID columns  
numerical_features = [f for f in numerical_features if f not in ['customer_id', 'is_high_value', 'churn_risk', 'recency_score', 'frequency_score', 'monetary_score']]  
# Prepare data for feature importance  
X = ml_features[numerical_features].fillna(0)  
y = ml_features['is_high_value']  
# Calculate feature importance using correlation  
feature_importance = []  
for feature in numerical_features:  
    correlation = np.corrcoef(X[feature], y)[0, 1]  
    feature_importance.append((feature, abs(correlation)))  
  
feature_importance.sort(key=lambda x: x[1], reverse=True)  
  
print("📈 Top 15 Most Important Features for High-Value Customer Prediction:")  
top_features = feature_importance[:15]  
  
fig, ax = plt.subplots(figsize=(12, 8))  
features, importance = zip(*top_features)  
y_pos = np.arange(len(features))  
ax.barh(y_pos, importance, color='skyblue')  
ax.set_yticks(y_pos)  
ax.set_yticklabels(features)  
ax.invert_yaxis()  
ax.set_xlabel('Absolute Correlation with High-Value Customer')  
ax.set_title('Feature Importance for Customer Value Prediction')  
 
plt.tight_layout()  
plt.show()  
 
# Display top features  
top_features_df = pd.DataFrame(top_features, columns=['Feature', 'Correlation'])  
display(top_features_df)
## 7. Feature Validation & Export"
print("💾 FEATURE VALIDATION & EXPORT")  
print("=" * 50)  
 
# Data quality check  
print("🔍 Data Quality Check:")  
print(f"• ML Features shape: {ml_features.shape}")  
print(f"• Missing values: {ml_features.isnull().sum().sum()}")  
print(f"• Duplicate rows: {ml_features.duplicated().sum()}")  
 
# Check feature distributions  
print(f"📊 Feature Distributions Summary:")  
numeric_summary = ml_features[numerical_features].describe()  
print(numeric_summary.loc[['mean', 'std', 'min', 'max']].T.head(10))  
 
# Save engineered features  
print("💾 Saving engineered features...")  
customer_features.to_csv('../data/processed/customer_features_engineered.csv', index=False)  
product_features.to_csv('../data/processed/product_features_engineered.csv', index=False)  
transaction_features.to_csv('../data/processed/transaction_features_engineered.csv', index=False)  
regional_features.to_csv('../data/processed/regional_features_engineered.csv', index=False)  
ml_features.to_csv('../data/processed/ml_ready_features.csv', index=False)  

feature_importance_df = pd.DataFrame(feature_importance, columns=['feature', 'importance'])  
feature_importance_df.to_csv('../data/processed/feature_importance.csv', index=False)  

print("✅ All engineered features saved to data/processed/")  

# Create feature documentation  
feature_docs = {  
    'customer_features': 'Customer behavioral and RFM features',  
    'product_features': 'Product performance and popularity features',  
    'transaction_features': 'Basket analysis and transaction patterns',  
    'regional_features': 'Geographic performance metrics',  
    'ml_ready_features': 'Comprehensive features for machine learning',  
    'feature_importance': 'Feature importance rankings'  
}  

feature_docs_df = pd.DataFrame(list(feature_docs.items()), columns=['feature_set', 'description'])  
feature_docs_df.to_csv('../data/processed/feature_documentation.csv', index=False)  
print("🎉 FEATURE ENGINEERING COMPLETED SUCCESSFULLY!")  
print("=" * 60)  
print("📚 Feature Sets Created:")  
for feature_set, description in feature_docs.items():  
    print(f"   • {feature_set}: {description}")  
 
print(f"🚀 Next Steps:")  
print("• Use ml_ready_features.csv for machine learning models")  
print("• Build predictive models for customer value and churn")  
print("• Create segmentation models using engineered features")  
print("• Integrate features into the Streamlit dashboard")  
print("• Use feature importance for business decision making")  
print("💡 Business Applications:")  
print("• High-value customer identification")  
print("• Churn prediction and prevention")  
print("• Product recommendation systems")  
print("• Regional expansion strategies")
print("• Personalized marketing campaigns")